# Gun Violence in the U.S. from 2013-2018

##### Team: Edward Chen, Andres Baez

## Project Overview:
***
- Reseach question, hypothesis
  - What do we hope to achieve?
- Datasets and Background & Prior Work
  - Brief summary of our datasets
  - Previous work that have been done using these datasets
- Data Cleaning and Analysis
  - Gun Violence Data 
- Discussion and Moving Forward
- Conclusion and Implications
  - What further questions we may have
  - How to improve our study


## Research Question
***
According to Google statistics, "gun violence in the United States results in tens of thousands of deaths and injuries annually". For this research project, we are interested in visualizing just how much gun violence, in particular mass shootings (+4 victims), there is in the U.S. over the course of the last 5 years and the relationship between gun violence and criminal activity, primarily in the state or county where there is the most amount of gun violence. We are also interested in the causes of mass shootings.

## Hypothesis
***
We predict throughout the past 5 years that overall gun violence in the U.S. should be decreasing with more awareness and occurrences of mass shootings. Therefore, we also predict that crime rates, in particular the number of homicides and other criminal activity regarding the use of guns, should also decrease throughout the years.


## Datasets:
***

> Dataset Name: Gun Violence Data - Comprehensive record of over 260k US gun violence incidents from 2013-2018
 Link to Dataset: https://www.kaggle.com/jameslko/gun-violence-data/kernels
 Number of Observations: 239,677

This dataset contains an accumulation of many different mass shooting (+4 victims) within the U.S., as well as additional information regarding each case location, age demographic, gender, and house and state affiliation. We plan on using this dataset to visualize the total extent of these mass shootings in the U.S. for the past 5 years, and consequently, discover which state or county has the most amount of mass shootings.


> Dataset Name: US Mass Shootings, 1982-2018: Data From Mother Jones’ Investigation
Link to Dataset: https://www.motherjones.com/politics/2012/12/mass-shootings-mother-jones-full-data/
Number of Observations: 108

This dataset contains an aggregation of famous mass shootings and a listing of the shooting location, number of casualties, mental health details of the suspect, weapon type, race, and gender. We plan to use this dataset to find correlations between potential causes of these mass shootings. 


>Dataset Name: Crimes in Chicago - An extensive dataset of crimes in Chicago (2001-2017), by City of Chicago
Link to Dataset: https://www.kaggle.com/currie32/crimes-in-chicago
Number of Observations: 7,941,282

This dataset contains an aggregation of crimes in Chicago from 2001 to 2017. We plan to primarily use this dataset to analyze the different levels of crime in the city and potentially find any correlation between gun violence and crime rates in Chicago.


## Background and Prior Work
***

## Methods
***
Data Cleaning:

Data Analysis:

What to report:

#### Data Cleaning and Analysis per Dataset
***

In [1]:
#import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from plotly import __version__
print ('plotly version: ', __version__) # requires version >= 1.9.0
from plotly.offline import init_notebook_mode, iplot, plot
import plotly.graph_objs as go
import plotly.plotly as py
import plotly.figure_factory as ff
import re
import collections

init_notebook_mode(connected=True)

import folium
from folium.plugins import HeatMap

plotly version:  3.4.2


In [ ]:
#Read in data from Gun Violence Archive
df = pd.read_csv('gv_data.csv')
print(df.describe())
print(df.head())

In [ ]:
#aggregate number of casualties
df['n_casualties'] =  df.apply(lambda row: row.n_killed + row.n_injured, axis=1)
print(df.head())

In [ ]:
#intializing global variables

#defining a color scheme
def hex_to_rgb(value):
    value = value.lstrip('#')
    lv = len(value)
    return tuple(int(value[i:i + lv // 3], 16) for i in range(0, lv, lv // 3))
print(hex_to_rgb("#472D7BFF"))
print(hex_to_rgb("#31688EFF"))
print(hex_to_rgb("#1F9A8AFF"))
print(hex_to_rgb("#5DC863FF"))
print(hex_to_rgb("#E3E418FF"))

#defining state codes
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY',
}

In [ ]:
#initial cleaning on data

#clean data to remove rows where lat and lon are NaN
df = df[np.isfinite(df['latitude']) & np.isfinite(df['longitude'])]
df.head()

In [ ]:
#Find all rows where the row contains a nan value (for state, state_house_district, staet senate district)
#YOUR CODE HERE:
null_rows_state=df[df['state'].isnull()]
null_rows_state_house_district=df[df['state_house_district'].isnull()]
null_rows_state_senate_district=df[df['state_senate_district'].isnull()]


In [ ]:
#separate data by year
#calculate the number of causalties, injured, and killed 
cas_by_year = {}
cas_by_year_month = {}
for date, n, k, i in zip(df['date'], df['n_casualties'], df['n_killed'], df['n_injured']):
    year = date[:4]
    year_month = date[:7]
    if year in cas_by_year:
        cas_by_year[year][0] += n
        cas_by_year[year][1] += k
        cas_by_year[year][2] += i
    else:
        cas_by_year[year] = [n, k, i]
        
    if year_month in cas_by_year_month:
        cas_by_year_month[year_month][0] += n
        cas_by_year_month[year_month][1] += k
        cas_by_year_month[year_month][2] += i
    else:
        cas_by_year_month[year_month] = [n, k, i]

years = list(cas_by_year.keys())
year_causalties = [item[0] for item in list(cas_by_year.values())]
year_killed = [item[1] for item in list(cas_by_year.values())]
year_injured = [item[2] for item in list(cas_by_year.values())]

total_per_year = {}
for date in df['date']:
    year = date[:4]
    if year in total_per_year:
        total_per_year[year] += 1
    else:
        total_per_year[year] = 1
        
data_per_year = list(total_per_year.values())

In [ ]:
#plot total number of data points per year 
data = [go.Bar(
    x=years,
    y=data_per_year,
    name='Number Killed per Year',
    marker=dict(
        color='rgb(117,107,177)'
    ),
    opacity=0.6
)]

layout = go.Layout(
    title='Total # of Data Points per Year',
    xaxis=dict(
        title='Year',
        titlefont=dict(
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='# Data Points',
        titlefont=dict(
            size=18,
            color='#7f7f7f'
        )
    )
)

fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [ ]:
#bar chart to  see total number of casualties between years
#separate casualties by total number killed and total number injured
trace1 = go.Bar(
    x=years,
    y=year_killed,
    name='Number Killed per Year',
    marker=dict(
        color='rgb(49, 104, 142)'
    ),
    opacity=0.6
)

trace2 = go.Bar(
    x=years,
    y=year_injured,
    name='Number Injured per Year',
    marker=dict(
        color='rgb(93, 200, 99)'
    ), 
    opacity=0.6
)

data = [trace1, trace2]

layout = go.Layout(
    barmode='stack',
    title='Total # of Casualties per Year',
    xaxis=dict(
        title='Year',
        titlefont=dict(
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='# of Casualties by Killed/Injured',
        titlefont=dict(
            size=18,
            color='#7f7f7f'
        )
    )
)
fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [ ]:
#plot total number of causalties by year, split by month
#split dictionary by year 
months = []
split_by_year = []

curr_data = []
curr_year = '2013'
curr_month = '00'

for k,v in cas_by_year_month.items():
    month = k[5:]
    if month not in months:
        months.append(month)
    year = k[:4]
    if year != curr_year:
        split_by_year.append(curr_data)
        curr_data = []
        curr_data.append(v)
        curr_year = year
        curr_month = '01'
    elif month != curr_month:
        curr_data.append(v)
        curr_month = month
split_by_year.append(curr_data)

cas_year_2013 = [item[0] for item in split_by_year[0]]
cas_year_2014 = [item[0] for item in split_by_year[1]]
cas_year_2015 = [item[0] for item in split_by_year[2]]
cas_year_2016 = [item[0] for item in split_by_year[3]]
cas_year_2017 = [item[0] for item in split_by_year[4]]
cas_year_2018 = [item[0] for item in split_by_year[5]]

In [ ]:
# Create traces
trace13 = go.Scatter(
    x = months,
    y = cas_year_2013,
    mode = 'lines+markers',
    name = 'Year 2013',
    opacity = 0.75,
    marker=dict(
        color='rgb(71, 45, 123)'
    )
)
trace14 = go.Scatter(
    x = months,
    y = cas_year_2014,
    mode = 'lines+markers',
    name = 'Year 2014',
    opacity = 0.75,
    marker=dict(
        color='rgb(49, 104, 142)'
    )
)
trace15 = go.Scatter(
    x = months,
    y = cas_year_2015,
    mode = 'lines+markers',
    name = 'Year 2015',
    opacity = 0.75,
    marker=dict(
        color='rgb(31, 154, 138)'
    )
)
trace16 = go.Scatter(
    x = months,
    y = cas_year_2016,
    mode = 'lines+markers',
    name = 'Year 2016',
    opacity = 0.75,
    marker=dict(
        color='rgb(227, 228, 24)'
    )
)
trace17 = go.Scatter(
    x = months,
    y = cas_year_2017,
    mode = 'lines+markers',
    name = 'Year 2017',
    opacity = 0.75,
    marker=dict(
        color='rgb(93, 200, 99)'
    )
)
trace18 = go.Scatter(
    x = months,
    y = cas_year_2018,
    mode = 'lines+markers',
    name = 'Year 2018',
    opacity = 0.75,
    marker=dict(
        color='rgb(71, 45, 123)'
    )
)

layout = go.Layout(
    title='Total # of Casualties per Month, split by year',
    xaxis=dict(
        title='Month',
        titlefont=dict(
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='# of Casualties by Killed+Injured',
        titlefont=dict(
            size=18,
            color='#7f7f7f'
        )
    )
)

data = [trace13, trace14, trace15, trace16, trace17, trace18]
fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [ ]:
#explore age demographic
unique_demographic = df.participant_age_group.unique()
cleanedList = [x for x in unique_demographic if str(x) != 'nan']
demographic = []
for elem in cleanedList:
    if not elem:
        continue
    split_elem = elem.replace('|',' ').replace(':',' ').split()
    for each in split_elem:
        if len(each) >=4 and each not in demographic:
            demographic.append(each)
demographic = [x for x in demographic if not any(x1.isdigit() for x1 in x)]
print(demographic)

#count total number of casualties in each demographic
count_demographic = {}
count_demographic[demographic[0]] = 0
count_demographic[demographic[1]] = 0
count_demographic[demographic[2]] = 0

for elem in cleanedList:
    if not elem:
        continue
    split_elem = elem.replace('|',' ').replace(':',' ').split()
    for each in split_elem:
        if demographic[0] in elem:
            count_demographic[demographic[0]] += 1
        elif demographic[1] in elem:
            count_demographic[demographic[1]] += 1
        elif demographic[2] in elem:
            count_demographic[demographic[2]] += 1
print(count_demographic)
demographic_counts = list(count_demographic.values())
print(demographic_counts)
edit_demographic = ['Adult: +18', 'Teen: 12-17', 'Child: 0-11']

In [ ]:
#plot total number of casualties split by age demographic 
data = [go.Bar(
    x=edit_demographic,
    y=demographic_counts,
    name='Number of Casualties by Age Group',
    marker=dict(
        color='rgb(117,107,177)'
    ),
    opacity=0.6
)]

layout = go.Layout(
    title='Number of Casualties by Age Demographic',
    xaxis=dict(
        title='Age Demographic',
        titlefont=dict(
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='# of Casualties',
        titlefont=dict(
            size=18,
            color='#7f7f7f'
        )
    )
)

fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [ ]:
#Use same process, find demographic of shooters based on age
#YOUR CODE HERE
df_age = df['participant_age'].dropna()
#for x in df_age:
#   print(x)

unique_demographic = df_age.unique()
#create list with non null values for participant age
cleanedList = [x for x in unique_demographic if str(x) != 'nan']
ages = {}

for elem in cleanedList:
    if not elem:
        continue
    split_elem = elem.replace('|',' ').replace(':',' ').split()
    #print(split_elem)
    split_elem = split_elem[1::2]
    for x in split_elem:
        x = int(x)
        if x in ages:
            ages[x] += 1
        else:
            ages[x] = 1
            
#print(ages)
od = collections.OrderedDict(sorted(ages.items()))
above_101 = []
for k,v in od.items():
    if k > 101:
        above_101.append(k)
for x in above_101:
    del od[x]
y_axis = list(od.values())
x_axis = list(od.keys())
print(od)

In [ ]:
data = [go.Bar(
    x=x_axis,
    y=y_axis,
    name='Number of Casualties by Age',
    marker=dict(
        color='rgb(49, 104, 142)'
    ),
    opacity=0.6
)]

layout = go.Layout(
    title='Number of Casualties by Age Demographic',
    xaxis=dict(
        title='Age Demographic',
        titlefont=dict(
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='# of Casualties',
        titlefont=dict(
            size=18,
            color='#7f7f7f'
        )
    )
)

fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [ ]:
#Use same process, find gender difference of casualties
#YOUR CODE HERE
df_gender = df['participant_gender'].dropna()


In [ ]:
#Find unique Participant Relationships
#Find total count for each unique value
#YOUR CODE HERE

In [ ]:
#aggregate density by state
cas_by_state = {}
for state, n, k, i in zip(df['state'], df['n_casualties'], df['n_killed'], df['n_injured']):
    if state == 'District of Columbia':
        continue
    if state in cas_by_state:
        cas_by_state[state][0] += n
        cas_by_state[state][1] += k
        cas_by_state[state][2] += i
    else:
        cas_by_state[state] = [n, k, i]
        
states = list(cas_by_state.keys())
casualties = [item[0] for item in list(cas_by_state.values())]
max_cas = max(casualties)

df_state = pd.DataFrame(states, columns=['states'])
df_state['n_casualties'] = casualties

df_state['cas_density'] = df_state.apply(lambda row: row.n_casualties/max_cas, axis=1)
df_state['code'] = df_state.apply(lambda row: us_state_abbrev[row.states], axis=1)


#get number killed and injured separated by state        
killed = [item[1] for item in list(cas_by_state.values())]
injured = [item[2] for item in list(cas_by_state.values())]

df_state['n_killed'] = killed
df_state['n_injured'] = injured
        
        
df_state['text'] = df_state.apply(lambda row: str(row.states) + '<br>' +\
                                  'Killed: '+str(row.n_killed)+
                                  ', Injured: '+str(row.n_injured), axis=1)


print(df_state.head())

In [ ]:
#choropleth map for gun casualties by state
scl = [[0.0, 'rgb(242,240,247)'],[0.2, 'rgb(218,218,235)'],[0.4, 'rgb(188,189,220)'],\
            [0.6, 'rgb(158,154,200)'],[0.8, 'rgb(117,107,177)'],[1.0, 'rgb(84,39,143)']]

data = [ dict(
        type='choropleth',
        colorscale = scl,
        autocolorscale = False,
        locations = df_state['code'],
        z = df_state['cas_density'].astype(float),
        text = df_state['text'],
        locationmode = 'USA-states',
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            ) ),
        colorbar = dict(
            title = "Casualties ~ max 16231")
        ) ]

layout = dict(
        title = '2013-2018 US Gun Violence Casualties by State<br>(Hover for breakdown)',
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showlakes = True,
            lakecolor = 'rgb(255, 255, 255)'),
             )
    
fig = dict( data=data, layout=layout )
iplot(fig)

In [ ]:
#count number of casualties > 4
len([x for x in df['n_casualties'] if x >= 4])

df_fips = df[df['n_casualties'] >= 4]
print(len(df_fips))


In [ ]:
import requests
import urllib

county_fips = []
'''
count = 0
for lat, lon, n in zip(df_fips['latitude'], df_fips['longitude'], df_fips['n_casualties']):
    if n >= 4:
        #Encode parameters 
        params = urllib.parse.urlencode({'latitude': lat, 'longitude': lon, 'format':'json'})
        #Contruct request URL
        url = 'https://geo.fcc.gov/api/census/block/find?' + params

        #Get response from API
        response = requests.get(url)

        #Parse json in response
        data = response.json()

        #Print FIPS code
        try:
            print(data['County']['FIPS'])
            county_fips.append(data['County']['FIPS'])
        except:
            county_fips.append(np.nan)
'''

#counties for 4 or more causalties per shooting
county_fips = ['42003', '06037', '39093', '08005', '37081', '40143', '35001', '22071', '06013', '24510', '47065', '29510', '22101', '11001', '39023', '47157', '06115', '17031', '22071', '06095', '10003', '49035', '06059', '40143', '26081', '06037', '13021', '22017', '13071', '26145', '06047', '18097', '28049', '29095', '11001', '06073', '36043', '06099', '06067', '06077', '12099', '17031', '29095', '36047', '06047', '53033', '45047', '36005', '06037', '42101', '06095', '04013', '21067', '39153', '53033', '17031', '22051', '17171', '39093', '12083', '37119', '42045', '47113', '34013', '37101', '06081', '42021', '06037', '18005', '42101', '34017', '04021', '22071', '39061', '26163', '42101', '51111', '47157', '26163', '17031', '47057', '06029', '26049', '51650', '47157', '17031', '13121', '55079', '06095', '18097', '51810', '13285', '06037', '42133', '29510', '17031', '37081', '44007', '47037', '48201', '17031', '17031', '51710', '37147', '24510', '44007', '47065', '06067', '29095', '22071', '51810', '29095', '17031', '06107', '17031', '45019', '36047', '08001', '48439', '17031', '26125', '36047', '01077', '17031', '28075', '06077', '12011', '24510', '17201', '37119', '06075', '21087', '39099', '11001', '40109', '34021', '26081', '20173', '06001', '06039', '09003', '36047', '36047', '20177', '26163', '54033', '12086', '48113', '53077', '35025', '18097', '34013', '26163', '06053', '29095', '42089', '34013', '48113', '29510', '10003', '36047', '51740', '42101', '40109', '06075', '34011', '39095', '45025', '17031', '17031', '24510', '45033', '27053', '12125', '06001', '37119', '29510', '18089', '09001', '36061', '47035', '11001', '37111', '08041', '13135', '53077', '11001', '06077', '26065', '32003', '12097', '47157', '17031', '06037', '48349', '12099', '29510', '26121', '20173', '54011', '37001', '39119', '17031', '31055', '06019', '36029', '04013', '42101', '48277', '40143', '42043', '12011', '12086', '42003', '09009', '12086', '04013', '48245', '06061', '37125', '06095', '45047', '21093', '06065', '06037', '11001', '12031', '26163', '48201', '39151', '04013', '36047', '48201', '48201', '42101', '40143', '06001', '27053', '36055', '18097', '06019', '13185', '26065', '12086', '20177', '10003', '37097', '48201', '12086', '34021', '37045', '26121', '36103', '34013', '22057', '22103', '01101', '39153', '36047', '51710', '17031', '26163', '36081', '17195', '13121', '39049', '42049', '28049', '41051', '48451', '22065', '17089', '01089', '26163', '06037', '45079', '06019', '40019', '49049', '06071', '12099', '19153', '51153', '34013', '13089', '24027', '12099', '17031', '37127', '53033', '55079', '18081', '48201', '17031', '12123', '12073', '22071', '39039', '12086', '40109', '06001', '06077', '18089', '12011', '32003', '48113', '26145', '12095', '18003', '48439', '12031', '18097', '12103', '06049', '47157', '06037', '06037', '51191', '12086', '17031', '26163', '42003', '47113', '18097', '45007', '34021', '06013', '17031', '39143', '55079', '48453', '47157', '36047', '37081', '45079', '13033', '48245', '36047', '37129', '45019', '39061', '12011', '29201', '48439', '47179', '06075', '06037', '37081', '06019', '06071', '22033', '45009', '37119', '28105', '45009', '48099', '26163', '06095', '04013', '17031', '25013', '40109', '22071', '17201', '47065', '18177', '48439', '22071', '11001', '34011', '40143', '27013', '27013', '17089', '12057', '17031', '06037', '48201', '47157', '17031', '13115', '36083', '17031', '13067', '17031', '05031', '48201', '17031', '08059', '48157', '47157', '12057', '06037', '22017', '18097', '13089', '13089', '06067', '06067', '47157', '32003', '17031', '13121', '13089', '11001', '06065', '39043', '13067', '18097', '18097', '48113', '48029', '06083', '22071', '06037', '45051', '26163', '37063', '39095', '06029', '06059', '18097', '51710', '06071', '06067', '17031', '17031', '13121', '17031', '45019', '17031', '26125', '45015', '32003', '11001', '12071', '01039', '17031', '34031', '47113', '06037', '47157', '37065', '17031', '17105', '55079', '51041', '18067', '17031', '13185', '17031', '24033', '11001', '47157', '06097', '48201', '12086', '25017', '55079', '26163', '47093', '41005', '06073', '36061', '06013', '06037', '22071', '29510', '48201', '21067', '42045', '51710', '17163', '26077', '51710', '41051', '48029', '48201', '18097', '22117', '29510', '12086', '36029', '48201', '06071', '21195', '21195', '17031', '44007', '48201', '17031', '06037', '39095', '11001', '53033', '09001', '06067', '06077', '17031', '17163', '32003', '47157', '34013', '42003', '17031', '13321', '05069', '47093', '23031', '29510', '26077', '36061', '42101', '37051', '42101', '42003', '48113', '55105', '34033', '25005', '48201', '27123', '47119', '29510', '51047', '13121', '37051', '06029', '17031', '55079', '20173', '55079', '35001', '11001', '42101', '27053', '06065', '11001', '13167', '22071', '48113', '36029', '13015', '47125', '22071', '47157', '17031', '47093', '49035', '34039', '25025', '17031', '36081', '06081', '32003', '37083', '44007', '22099', '47157', '45031', '13245', '06037', '17031', '17031', '40143', '26163', '12053', '01073', '06001', '17043', '37081', '40143', '06071', '28049', '36055', '06037', '12019', '26049', '12095', '13021', '06037', '45019', '26163', '02020', '51710', '48355', '22071', '12041', '36081', '28107', '39029', '42101', '45031', '29510', '42101', '18097', '12086', '45029', '06019', '26163', '28095', '28095', '47157', '37107', '06037', '17031', '12031', '26163', '13121', '51760', '36001', '40109', '17031', '36065', '06077', '13121', '13121', '13135', '26163', '12127', '36081', '36047', '45003', '37155', '26163', '06067', '53061', '21221', '47157', '53033', '06075', '06037', '06037', '06037', '51810', '37081', '37137', '29077', '12086', '39153', '48141', '32003', '17031', '12073', '34005', '42003', '48029', '46109', '39035', '39023', '08031', '51001', '06075', '48491', '47157', '34013', '13121', '47065', '36047', '54061', '29510', '51700', '37025', '48029', '18141', '13021', '41051', '12057', '12086', '42091', '17031', '42045', '34039', '17201', '28153', '17031', '12115', '48121', '26163', '13261', '36055', '17031', '12086', '21049', '29510', '17163', '06067', '06037', '22071', '37119', '47157', '06099', '13051', '51770', '48113', '13223', '12086', '47065', '25025', '06075', '20091', '20173', '16057', '12105', '06085', '37051', '17201', '51740', '48029', '47125', '25025', '36081', '31055', '06077', '13089', '37147', '13285', '36067', '36061', '37169', '39035', '40143', '13097', '47033', '12101', '21235', '46083', '06037', '05119', '06067', '13137', '45019', '48027', '12127', '48201', '29215', '24510', '26163', '12095', '34001', '32003', '29510', '47093', '45073', '17031', '13215', '45007', '17089', '48463', '28085', '13121', '06065', '01069', '06037', '37085', '48373', '06077', '04013', '34013', '04001', '48113', '40109', '12057', '02122', '35001', '47125', '18097', '48375', '12005', '40143', '24510', '12127', '22103', '28091', '42003', '21111', '18097', '26021', '13115', '04013', '36063', '34031', '01101', '42081', '37119', '37155', '51760', '13261', '34021', '36055', '36047', '04013', '55079', '39113', '48201', '18141', '36005', '55139', '36029', '26163', '39061', '39035', '34017', '34013', '04019', '24033', '29047', '22071', '02020', '24510', '36055', '17201', '55079', '48309', '12086', '29189', '24005', '06019', '01101', '32003', '29510', '26049', '25023', '17115', '22071', '42045', '31055', '17031', '45091', '06073', '13247', '39035', '09009', '24033', '36103', '19163', '22071', '17031', '36029', '30077', '12127', '29510', '06037', '39061', '09001', '48201', '12086', '13009', '36005', '13113', '40109', '39049', '48113', '34007', '29510', '40143', '36047', '12086', '45019', '44007', '26163', '42101', '37007', '49057', '21067', '42003', '42101', '37193', '17031', '22099', '36061', '22071', '26163', '09003', '12099', '25005', '26163', '10001', '17119', '12086', '42101', '11001', '36067', '21111', '25003', '48029', '17031', '22017', '18003', '34013', '45091', '24510', '39035', '22033', '34017', '17031', '06077', '06037', '29510', '39035', '45075', '24510', '13121', '26163', '47065', '23019', '48113', '39061', '17031', '22033', '41047', '06073', '06111', '37065', '51800', '21111', '36005', '12086', '13117', '29510', '22055', '42049', '51670', '22071', '18097', '29095', '17201', '17055', '42003', '55079', '12095', '17031', '24510', '17031', '13051', '36047', '36047', '29510', '45079', '50023', '05093', '37071', '48201', '28123', '29095', '26163', '06037', '42003', '06037', '45075', '45069', '17031', '48439', '36055', '48427', '39061', '37063', '35005', '12095', '06099', '22071', '17031', '51067', '27053', '12099', '06053', '36047', '47163', '13021', '48423', '47157', '34025', '37119', '12127', '32003', '29019', '18089', '08031', '53007', '17031', '13071', '27029', '37119', '22017', '36055', '27053', '27053', '22033', '35061', '12083', '22033', '34013', '26025', '46023', '18097', '40143', '17031', '42101', '06053', '22017', '13135', '12071', '17031', '17183', '29095', '17031', '13199', '17031', '39113', '17031', '39061', '37155', '12075', '41019', '24510', '24510', '04005', '17143', '37119', '47157', '13089', '15003', '18039', '12071', '22071', '17031', '47037', '44007', '45009', '04013', '37101', '11001', '21199', '34013', '18163', '48439', '37021', '22109', '08041', '45007', '12031', '48201', '23011', '06029', '08031', '26099', '19153', '18097', '06037', '12031', '42021', '42101', '01019', '06019', '42003', '24510', '36071', '45019', '17031', '48061', '53033', '22071', '48201', '39049', '27053', '45051', '25025', '06067', '08041', '17091', '40031', '13051', '06071', '31055', '22057', '39061', '12073', '42003', '24510', '13051', '17163', '06037', '06059', '41039', '12086', '12086', '37129', '06001', '12031', '01097', '42101', '47113', '06037', '22071', '12105', '47157', '17031', '11001', '10003', '13089', '51073', '40091', '28109', '06037', '12091', '06065', '53033', '51550', '51033', '08031', '04013', '22071', '11001', '06037', '48463', '12057', '12095', '28047', '36055', '12095', '17031', '26121', '13081', '27053', '22051', '01063', '48201', '06037', '49035', '06095', '29165', '12057', '26077', '12095', '01073', '28141', '29189', '48201', '04013', '12127', '13121', '08093', '20079', '53045', '51153', '26163', '06065', '12111', '39049', '20173', '06037', '13121', '25025', '20209', '22055', '48029', '42003', '26163', '34021', '13105', '06001', '12083', '41051', '12071', '17031', '13121', '12071', '12011', '01051', '21111', '17031', '48181', '06019', '17031', '08031', '01077', '22017', '32003', '17031', '35001', '12009', '17031', '47157', '01015', '35001', '06037', '55079', '26163', '01031', '12095', '45045', '48215', '42101', '06037', '26163', '17031', '17031', '01009', '48157', '24510', '13073', '39001', '53033', '32003', '01081', '36061', '20177', '17031', '51083', '08031', '12086', '32003', '47037', '47037', '24033', '22033', '28097', '06001', '29189', '47149', '12019', '27053', '12086', '32003', '22015', '26163', '01101', '24031', '48453', '17031', '01073', '17031', '13121', '54039', '13071', '18163', '45019', '18097', '24510', '13035', '06077', '47037', '22071', '34013', '36061', '25017', '53077', '32003', '48201', '34021', '18097', '06067', '24510', '17031', '18003', '08031', '01005', '12071', '04013', '36047', '27053', '06107', '25025', '11001', '12091', '35005', '27131', '06077', '06037', '12095', '36047', '17031', '06019', '13063', '06001', '10003', '51131', '39155', '17031', '13299', '51770', '34005', '21111', '53067', '17031', '13089', '24033', '09003', '20209', '48439', '51710', '17031', '17031', '13089', '45069', '45083', '37119', '48201', '48355', '41047', '17031', '06037', '32003', '17031', '22073', '47065', '36047', '48201', '39035', '04013', '17031', '42003', '06037', '13223', '48113', '22033', '47163', '26075', '26021', '11001', '24510', '13153', '53067', '29510', '48201', '06001', '39153', '53015', '39035', '06029', '48113', '06071', '47125', '26163', '48201', '22033', '05035', '06019', '17031', '17091', '39061', '36047', '45019', '48021', '39017', '12071', '17031', '36015', '24510', '13059', '29510', '13191', '53061', '29189', '48453', '12086', '18163', '12011', '13089', '47157', '06065', '26125', '42011', '17031', '08001', '28149', '06037', '39035', '12031', '35001', '17031', '06067', '42101', '12031', '48113', '55079', '29097', '36081', '06037', '51710', '27053', '06001', '09009', '29095', '06037', '53053', '01097', '09009', '38089', '25025', '09001', '34011', '01073', '12086', '48041', '36005', '48303', '29510', '37051', '01123', '06065', '34001', '13121', '17031', '37163', '17031', '37129', '19153', '51770', '17031', '36047', '42011', '48453', '05051', '12086', '01073', '37049', '26145', '29095', '22071', '18003', '34017', '18003', '48201', '56021', '48439', '06037', '42101', '11001', '18097', '12086', '12095', '42003', '42101', '51680', '48201', '47037', '53057', '24510', '17019', '37067', '48201', '48201', '06087', '45007', '06075', '06019', '17091', '48029', '27053', '22071', '36047', '26081', '06067', '17031', '48113', '17031', '01097', '06037', '17201', '17031', '06075', '01089', '06001', '40081', '51770', '39113', '13151', '12086', '28049', '39113', '06065', '06037', '39095', '24033', '36071', '36061', '22017', '47157', '05119', '34013', '17031', '42101', '06001', '17031', '12011', '48029', '45051', '17031', '47157', '53033', '48201', '42125', '48201', '16001', '09001', '29095', '06067', '06073', '48029', '12031', '12095', '47045', '17031', '06037', '12051', '28033', '36001', '21111', '02090', '22071', '17031', '22071', '29095', '06037', '48215', '06037', '24510', '06033', '34013', '06037', '18039', '35001', '21067', '48201', '06019', '36047', '37065', '12095', '24510', '06067', '22055', '37119', '17031', '17031', '37119', '17031', '48029', '37187', '01073', '17031', '37195', '37157', '36119', '17031', '01045', '13215', '39139', '01101', '09009', '48201', '13277', '12086', '28011', '48113', '45063', '17201', '41047', '48085', '06071', '06037', '12011', '26049', '21205', '08041', '17031', '34013', '06053', '29095', '21235', '28113', '17031', '08021', '48201', '12086', '28107', '06019', '47157', '12099', '13063', '13121', '48039', '17031', '48029', '17031', '13095', '53035', '47075', '48215', '04013', '24033', '22017', '27053', '47157', '11001', '12099', '28163', '42003', '40111', '05119', '45079', '39035', '28049', '22071', '47071', '17031', '36071', '12079', '29155', '06053', '17031', '36007', '27123', '42101', '18141', '51710', '16041', '19169', '55025', '39035', '06037', '28075', '39035', '36055', '20079', '12031', '39155', '18089', '51700', '39061', '06065', '42101', '48165', '20177', '06067', '06025', '44007', '13215', '17031', '27053', '17089', '22071', '17031', '24005', '22051', '48453', '26163', '21111', '48439', '55073', '22001', '24017', '29510', '06019', '42043', '39035', '26163', '42077', '47093', '39035', '39061', '48201', '12117', '39151', '17031', '12057', '40109', '06013', '17031', '19153', '48303', '42101', '22019', '27123', '17031', '48201', '37035', '18003', '48029', '06071', '13121', '18097', '17195', '06065', '01073', '17031', '42101', '06095', '37139', '17201', '39049', '53033', '21111', '12009', '22071', '13157', '12086', '34013', '42045', '09003', '06037', '51810', '24031', '06037', '06067', '06037', '20177', '06073', '12031', '45051', '48113', '12031', '26049', '17031', '20209', '48201', '34021', '39089', '17031', '05031', '12055', '48439', '45029', '42101', '04013', '19153', '06015', '12086', '11001', '36061', '28085', '01113', '28059', '34031', '21111', '39017', '29510', '48479', '06037', '29510', '22071', '22071', '18175', '12095', '06019', '49035', '42131', '22071', '48439', '47157', '47065', '48201', '17031', '29189', '24510', '24510', '06075', '39151', '51510', '35039', '51760', '36047', '17031', '39049', '11001', '29510', '17031', '45051', '01097', '13121', '37147', '36061', '12011', '06073', '34005', '29510', '11001', '17031', '12086', '39153', '36005', '34013', '17091', '37067', '05119', '06001', '34017', '45047', '26125', '01001', '51650', '12095', '39071', '37183', '13051', '23025', '17031', '28075', '42017', '39061', '17119', '51025', '06065', '01073', '45077', '17031', '42101', '55079', '34021', '04013', '06037', '39049', '48201', '51760', '39061', '22033', '17031', '12031', '37081', '17031', '10003', '36029', '37159', '48141', '17115', '08041', '17031', '06019', '53077', '24510', '18141', '11001', '12091', '30003', '12073', '45005', '34029', '06001', '36005', '36005', '17031', '29189', '42101', '06085', '06067', '42077', '51800', '06037', '29510', '36103', '48029', '39035', '17031', '36047', '34013', '24510', '34013', '47157', '37083', '17031', '06071', '29189', '06067', '18163', '35009', '06067', '17031', '17031', '21111', '06037', '17031', '17031', '01047', '48085', '22071', '47157', '42101', '42133', '53063', '45079', '51730', '26163', '18089', '17183', '06029', '09003', '09009', '24510', '47037', '36067', '47157', '42101', '24510', '22071', '06037', '47157', '20045', '12115', '12086', '06001', '04021', '17031', '48439', '04013', '29019', '06075', '06085', '39087', '06059', '04013', '48201', '28075', '36047', '24025', '08069', '42045', '12057', '47037', '12031', '09003', '01017', '06075', '21111', '39095', '51730', '48453', '18089', '27053', '36029', '06037', '22103', '06029', '39099', '48453', '06037', '48493', '48113', '49049', '42101', '39113', '04013', '48113', '22071', '37051', '18089', '06019', '37107', '13121', '06103', '06013', '13215', '22045', '27053', '42033', '39035', '28085', '26163', '26163', '34017', '29510', '47157', '42003', '17031', '37119', '25025', '06077', '48113', '21111', '39049', '06037', '28007', '26125', '13215', '22055', '42101', '22097', '01073', '28149', '01109', '04013', '12086', '35001', '47157', '42003', '32003', '08035', '17031', '34025', '01089', '05095', '17031', '28035', '06037', '01011', '06089', '12011', '48167', '29169', '37119', '47037', '01089', '01063', '12001', '45091', '11001', '01015', '42043', '17031', '12095', '39097', '42101', '21157', '13093', '11001', '12083', '06037', '21227', '42051', '42011', '18097', '29510', '39035', '08041', '12099', '13151', '21115', '26163', '22071', '12011', '26125', '20209', '47157', '48029', '04013', '12099', '26163', '09001', '36047', '12086', '17201', '29083', '01113', '06055', '37007', '18141', '26145', '17019', '04013', '06099', '12003', '53053', '36047', '55079', '17031', '21111', '17021', '06075', '49035', '48085', '34007', '34025']


In [ ]:
#check county codes 
df_fips['fips'] = county_fips

#aggregate joint counties
dict_fips = {}
for f, n in zip(df_fips['fips'], df_fips['n_casualties']):
    if f in dict_fips:
        dict_fips[f] += n
    else:
        dict_fips[f] = n

In [ ]:
#split data by county 
#aggregate density by state
#choropleth map split by county 
colorscale = ["#f7fbff","#ebf3fb","#deebf7","#d2e3f3","#c6dbef","#b3d2e9","#9ecae1",
              "#85bcdb","#6baed6","#57a0ce","#4292c6","#3082be","#2171b5","#1361a9",
              "#08519c","#0b4083","#08306b"]

endpts = list(np.linspace(1, 12, len(colorscale) - 1))
fips = list(dict_fips.keys())
values = list(dict_fips.values())

fig = ff.create_choropleth(
    fips=fips, values=values, scope=['usa'],
    binning_endpoints=endpts, colorscale=colorscale,
    show_state_data=False,
    show_hover=True, centroid_marker={'opacity': 1},
    asp=2.9, title='USA Casualties %',
    legend_title='Casualties %'
)
iplot(fig, filename='choropleth_full_usa')


In [ ]:
#plot density map on U.S. for number of casualities by long/lat
'''
limits = [(0,2),(3,10),(11,20),(21,50),(50,200)]
colors = ["rgb(0,116,217)","rgb(255,65,54)","rgb(133,20,75)","rgb(255,133,27)","lightgrey"]
density_cas = []
scale = 1

for i in range(len(limits)):
    lim = limits[i]
    df_sub = df[(df['n_casualties'] >= lim[0]) & (df['n_casualties'] <= lim[1])]
    cas = dict(
        type = 'scattergeo',
        locationmode = 'USA-states',
        lon = df_sub['longitude'],
        lat = df_sub['latitude'],
        text = "Number of guns involved: "+str(df_sub['n_guns_involved']),
        marker = dict(
            size = df_sub['n_casualties']/scale,
            # sizeref = 2. * max(df_sub['pop']/scale) / (25 ** 2),
            color = colors[i],
            line = dict(width=0.5, color='rgb(40,40,40)'),
            sizemode = 'area'
        ),
        name = '{0} - {1}'.format(lim[0],lim[1]) )
    density_cas.append(cas)

layout = dict(
        title = '2013-2018 Gun Violence<br>(Click legend to toggle traces)',
        showlegend = True,
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showland = True,
            landcolor = 'rgb(217, 217, 217)',
            subunitwidth=1,
            countrywidth=1,
            subunitcolor="rgb(255, 255, 255)",
            countrycolor="rgb(255, 255, 255)"
        ),
    )

fig = dict(data=density_cas, layout=layout)
iplot(fig, show_link=False)
'''

## Ethics and Privacy

## Dicussion

## Moving Forward